In [1]:
import os
import sys 
sys.path.append(r'C:\Users\ELAFACRB1\Codice\GitHub\rio-utils-app\src')
from utils.database_managers import QDrantDBManager
from utils.embedding import EmbeddingFunction
from utils.text_processing import TextSplitter
from utils.language_models import LangChainAI

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding = EmbeddingFunction('fast-bgeEmbedding').embedder

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [3]:
text_splitter = TextSplitter(
    chunk_size=2000, 
    chunk_overlap=20
)

In [4]:

QDRANT_URL=os.getenv('QDRANT_URL')
COLLECTION_NAME="csv"

qdrantClient = QDrantDBManager(
    url=QDRANT_URL,
    port=6333,
    collection_name=COLLECTION_NAME,
    vector_size=768,
    embedding=embedding,
    record_manager_url="sqlite:///record_manager_cache.sql"
)


Collection csv already exists!


c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [5]:
qdrantClient.vector_store

In [6]:
import pandas as pd

In [7]:
FILENAME=r"C:\Users\ELAFACRB1\Codice\GitHub\langchain-rag-chatbot\data\Adidas final.csv"

In [8]:
df = pd.read_csv(FILENAME, sep=';')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   URL            2625 non-null   object
 1   Product Name   2625 non-null   object
 2   Product ID     2625 non-null   object
 3   Listing Price  2625 non-null   int64 
 4   Sale Price     2625 non-null   int64 
 5   Discount       2625 non-null   int64 
 6   Brand          2625 non-null   object
 7   Description    2625 non-null   object
 8   Rating         2625 non-null   int64 
 9   Reviews        2625 non-null   int64 
 10  Images         2625 non-null   object
 11  Last Visited   2625 non-null   object
dtypes: int64(5), object(7)
memory usage: 246.2+ KB


In [10]:
#create an enhanced description for each row.
def extended_descriptin(row):
    ex_desc = f"Poduct Name: {row['Product Name'].strip()} \nDescription of product: {row['Description'].strip()}"
    return ex_desc
df['ExtendedDescription'] = df.apply(extended_descriptin, axis=1)
texts = df['ExtendedDescription'].to_list()
print(texts[0])

Poduct Name: Women's adidas Originals NMD_Racer Primeknit Shoes 
Description of product: Channeling the streamlined look of an '80s racer, these shoes are updated with modern features. The foot-hugging adidas Primeknit upper offers a soft, breathable feel. The Boost midsole provides responsive comfort accented with a contrast-color EVA heel plug. Embroidered details add a distinctive finish.


In [11]:
langchain_client=LangChainAI()
docs=langchain_client.create_documents(texts, {"source": "csv"})
docs

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


[Document(page_content="Poduct Name: Women's adidas Originals NMD_Racer Primeknit Shoes \nDescription of product: Channeling the streamlined look of an '80s racer, these shoes are updated with modern features. The foot-hugging adidas Primeknit upper offers a soft, breathable feel. The Boost midsole provides responsive comfort accented with a contrast-color EVA heel plug. Embroidered details add a distinctive finish.", metadata={'source': 'csv'}),
 Document(page_content="Poduct Name: Women's adidas Originals Sleek Shoes \nDescription of product: A modern take on adidas sport heritage, tailored just for women. Perforated 3-Stripes on the leather upper of these shoes offer a sleek look that mirrors iconic tennis styles.", metadata={'source': 'csv'}),
 Document(page_content="Poduct Name: Women's adidas Swim Puka Slippers \nDescription of product: These adidas Puka slippers for women's come with slim straps for a great fit. Feature performance logo on the footbed and textured Rubber outsole

In [12]:
splitted_docs = text_splitter.fixed_split(docs)

In [13]:
qdrantClient.index_documents(splitted_docs)

In [14]:
print("Documents loaded!")

Documents loaded!
